In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from torchvision import datasets, transforms
import pandas as pd
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import StratifiedShuffleSplit
import numpy as np

#### Reading the data

In [2]:
data = pd.read_csv('./data/clean/open_shot.csv')

In [3]:
pd.set_option('display.max_columns', None)

In [4]:
data.head()

,game_id,matchup,location,win,final_margin,shot_number,period,game_clock,shot_clock,dribbles,touch_time,shot_dist,pts_type,shot_result,closest_defender,closest_defender_id,close_def_dist,fgm,pts,player_name,player_id,defender_wingspan,close_def_dist_in,open_shot
0,21400899,"MAR 04, 2015 - CHA @ BKN",A,W,24,1,1,1:09,10.8,2,1.9,7.7,2,made,"Anderson, Alan",101187,1.3,1,2,brian roberts,203148,81.50,15.6,0
1,21400899,"MAR 04, 2015 - CHA @ BKN",A,W,24,2,1,0:14,3.4,0,0.8,28.2,3,missed,"Bogdanovic, Bojan",202711,6.1,0,0,brian roberts,203148,82.76,73.2,1
2,21400899,"MAR 04, 2015 - CHA @ BKN",A,W,24,3,1,0:00,NaN,3,2.7,10.1,2,missed,"Bogdanovic, Bojan",202711,0.9,0,0,brian roberts,203148,82.76,10.8,0
3,21400899,"MAR 04, 2015 - CHA @ BKN",A,W,24,4,2,11:47,10.3,2,1.9,17.2,2,missed,"Brown, Markel",203900,3.4,0,0,brian roberts,203148,80.75,40.8,1
4,21400899,"MAR 04, 2015 - CHA @ BKN",A,W,24,5,2,10:34,10.9,2,2.7,3.7,2,missed,"Young, Thaddeus",201152,1.1,0,0,brian roberts,203148,83.50,13.2,0


In [5]:
data.shape

(128069, 24)

In [6]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 128069 entries, 0 to 128068
Data columns (total 24 columns):
 #   Column               Non-Null Count   Dtype  
---  ------               --------------   -----  
 0   game_id              128069 non-null  int64  
 1   matchup              128069 non-null  object 
 2   location             128069 non-null  object 
 3   win                  128069 non-null  object 
 4   final_margin         128069 non-null  int64  
 5   shot_number          128069 non-null  int64  
 6   period               128069 non-null  int64  
 7   game_clock           128069 non-null  object 
 8   shot_clock           122502 non-null  float64
 9   dribbles             128069 non-null  int64  
 10  touch_time           128069 non-null  float64
 11  shot_dist            128069 non-null  float64
 12  pts_type             128069 non-null  int64  
 13  shot_result          128069 non-null  object 
 14  closest_defender     128069 non-null  object 
 15  closest_defender_

In [7]:
data['location'].value_counts()

location
A    64135
H    63934
Name: count, dtype: int64

In [8]:
data['win'].value_counts()

win
W    64595
L    63474
Name: count, dtype: int64

In [9]:
len(data['player_name'].value_counts())

281

#### Feature engineering

In [10]:
# Dropping unwanted columns 
data.drop(columns=['game_id', 'matchup', 'location', 'win'], inplace=True)

In [11]:
# To split the game clock in minutes and seconds
data[['minutes', 'seconds']] = data['game_clock'].str.split(':', expand=True)

# Removing the game_clock column
data.drop('game_clock', axis=1, inplace=True)

# Changing the data type of the column 
data['minutes'] = data['minutes'].astype(int)
data['seconds'] = data['seconds'].astype(int)

In [12]:
# For each player creating offensive and defensive score

# Finding the defense score

def calDefenseScore(groupedDf):
    return (groupedDf['shot_result'] == 'missed').sum() / groupedDf['shot_result'].count()

def calAttackScore(groupedDf):
    return (groupedDf['shot_result'] == 'made').sum() / groupedDf['shot_result'].count()

defenseResult = data.groupby('closest_defender_id').apply(calDefenseScore).to_frame() 
attackResult = data.groupby('player_id').apply(calAttackScore).to_frame()

defenseResult = defenseResult.reset_index().rename(columns={0: 'Defense_Score', 'closest_defender_id': 'player_id'})
attackResult = attackResult.rename(columns={0: 'Attack Score'}).reset_index()


In [13]:
combined_score_result =  pd.merge(defenseResult, attackResult, on='player_id')

In [14]:
# Appending the score to the dataframe

def defenderScore(defenderID):
    return defenseResult[defenseResult['player_id'] == int(defenderID)]['Defense_Score'].iloc[0]

def attackScore(attackID):
    return attackResult[attackResult['player_id'] == int(attackID)]['Attack Score'].iloc[0]


data['defense_player_score'] = data['closest_defender_id'].apply(defenderScore)
data['attack_player_score'] = data['player_id'].apply(attackScore)


In [15]:
# dropping unwanted columns
data.drop(columns=['player_id', 'player_name', 'closest_defender', 'closest_defender_id'], inplace=True)

In [16]:
# Final data set
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 128069 entries, 0 to 128068
Data columns (total 19 columns):
 #   Column                Non-Null Count   Dtype  
---  ------                --------------   -----  
 0   final_margin          128069 non-null  int64  
 1   shot_number           128069 non-null  int64  
 2   period                128069 non-null  int64  
 3   shot_clock            122502 non-null  float64
 4   dribbles              128069 non-null  int64  
 5   touch_time            128069 non-null  float64
 6   shot_dist             128069 non-null  float64
 7   pts_type              128069 non-null  int64  
 8   shot_result           128069 non-null  object 
 9   close_def_dist        128069 non-null  float64
 10  fgm                   128069 non-null  int64  
 11  pts                   128069 non-null  int64  
 12  defender_wingspan     127495 non-null  float64
 13  close_def_dist_in     128069 non-null  float64
 14  open_shot             128069 non-null  int64  
 15  

#### Data Preprocessing

In [17]:
# Imputing the data
imputer_median = SimpleImputer(strategy='median')
columns_to_impute = ['shot_clock', 'defender_wingspan']
data[columns_to_impute] = imputer_median.fit_transform(data[columns_to_impute])

In [18]:
# Normalizing the data
numerical_columns = data.select_dtypes(include=['number']).columns.tolist()
scaler = StandardScaler()
data[numerical_columns] = scaler.fit_transform(data[numerical_columns])

In [19]:
# One hot encoding the dataframe
def encodeShotResult(shortResult):
    if(shortResult == 'made'):
        return 1
    else:
        return 0
data['shot_result_encoded'] = data['shot_result'].apply(encodeShotResult)

# Dropping the shot_result column
data.drop(columns=['shot_result'], inplace=True)

#### Defining the model

In [20]:
# Define a simple neural network for classification
class SimpleClassifier(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(SimpleClassifier, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(hidden_size, output_size)
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        x = self.fc1(x)
        x = self.relu(x)
        x = self.fc2(x)
        x = self.sigmoid(x)
        return x

# Set random seed for reproducibility
torch.manual_seed(42)

#### Defining the hyper parameters

In [21]:
# Define hyperparameters
input_size = 18  # For example, if working with 28x28 images
hidden_size = 128
output_size = 1  # For binary classification

learning_rate = 0.001
batch_size = 64
epochs = 50

#### Loading the Train and Test data 

In [22]:
# Splitting the data into train and test.
split = StratifiedShuffleSplit(n_splits=1, test_size=0.2, random_state=42)

for train_index, test_index in split.split(data, data['shot_result_encoded']): 
    strat_train_set = data.iloc[train_index]
    strat_test_set = data.iloc[test_index]

In [23]:
# Defining the train and test dataset
train_data = strat_train_set.drop(columns=['shot_result_encoded'])
train_labels = strat_train_set[['shot_result_encoded']]

test_data = strat_test_set.drop(columns=['shot_result_encoded'])
test_labels = strat_test_set[['shot_result_encoded']]

In [24]:
# Converting to Tensors
X_train_tensor = torch.tensor(train_data.to_numpy(), dtype=torch.float32)
y_train_tensor = torch.tensor(train_labels.to_numpy(), dtype=torch.float32)

X_test_tensor = torch.tensor(test_data.to_numpy(), dtype=torch.float32)
y_test_tensor = torch.tensor(test_labels.to_numpy(), dtype=torch.float32)

In [25]:
train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
test_dataset = TensorDataset(X_test_tensor, y_test_tensor)

train_loader = DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(dataset=test_dataset, batch_size=batch_size, shuffle=True)

#### Initilizing the model

In [26]:
# Initialize model, loss function, and optimizer
model = SimpleClassifier(input_size, hidden_size, output_size)
criterion = nn.BCELoss()  # Binary Cross Entropy Loss for binary classification
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

#### Training the model

In [27]:
# Training loop
test_accuracy_list = [0]
threshold = 5
count = 0

for epoch in range(epochs):
    loss_list = []
    for inputs, labels in train_loader:
        inputs = inputs.view(-1, input_size) 
        outputs = model(inputs)
        loss = criterion(outputs, labels.float().view(-1, 1))

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        loss_list.append(loss.item())

    print('----------------------------------------------')
    print(f'Epoch {epoch+1}/{epochs}, Loss: {np.mean(loss_list)}')

    model.eval()

    correct = 0
    total = 0

    with torch.no_grad():
        for inputs, labels in test_loader:
            outputs = model(inputs)
            output_flatten = outputs.view(-1)
            predicted = (output_flatten > 0.5).float()
            total += labels.size(0)
            labels_flatten = labels.view(-1)
            correct += (predicted == labels_flatten).sum().item()

    accuracy = correct / total
    print(f'Test Accuracy: {accuracy}')

    if(accuracy < test_accuracy_list[-1]):
        count += 1
    else:
        count = 0
        test_accuracy_list.append(accuracy)
        torch.save(model, 'model.pth')
    
    if(count >= threshold):
        break

----------------------------------------------
Epoch 1/50, Loss: 0.029882068510061793
Test Accuracy: 1.0
----------------------------------------------
Epoch 2/50, Loss: 0.00010317227857969866
Test Accuracy: 1.0
----------------------------------------------
Epoch 3/50, Loss: 2.5199164391404685e-05
Test Accuracy: 1.0
----------------------------------------------
Epoch 4/50, Loss: 8.464369322784824e-06
Test Accuracy: 1.0
----------------------------------------------
Epoch 5/50, Loss: 3.177882583442396e-06
Test Accuracy: 1.0
----------------------------------------------
Epoch 6/50, Loss: 1.2532643646116142e-06
Test Accuracy: 1.0
----------------------------------------------
Epoch 7/50, Loss: 5.067759574854264e-07
Test Accuracy: 1.0
----------------------------------------------
Epoch 8/50, Loss: 2.0816828422153677e-07
Test Accuracy: 1.0
----------------------------------------------
Epoch 9/50, Loss: 8.580159688190909e-08
Test Accuracy: 1.0
-------------------------------------------